In [28]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm

# Custom functions to create visual charts
from chart_utils import TimeSeriesImageDataset, create_area_chart, create_bar_chart, accuracy_fn

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Read UCR Dataset
def read_ucr(filename):
    data = []
    labels = []
    
    with open(filename, 'r') as file:
        for line in file:
            parts = line.strip().split(',')
            if len(parts) < 2:  # Ensure there's at least one feature and one label
                continue
            features = [float(f) for f in parts[:-1]]
            label = int(parts[-1].split(':')[-1])  # Handle label after the colon
            data.append(features)
            labels.append(label)
    
    print(f"Loaded {len(data)} samples from {filename}")
    return np.array(data), np.array(labels)

# File paths
train_file = 'ECG/ECG_TRAIN.ts'
test_file = 'ECG/ECG_TEST.ts'

# Load dataset
x_train, y_train = read_ucr(train_file)
x_test, y_test = read_ucr(test_file)

# Normalize labels to be within range [0, num_classes-1]
nb_classes = len(np.unique(y_test))
y_train = (y_train - y_train.min()).astype(int)
y_test = (y_test - y_test.min()).astype(int)

# Normalize features
x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_train = (x_train - x_train_mean) / x_train_std
x_test = (x_test - x_train_mean) / x_train_std

# Create dataset and dataloader
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2)
])

train_dataset = TimeSeriesImageDataset(x_train, y_train, transform)
test_dataset = TimeSeriesImageDataset(x_test, y_test, transform)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define SimpleCNN
class SimpleCNN(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(SimpleCNN, self).__init__()
        self.layer_stack = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.layer_stack(x)

# Example usage of the model
input_shape = (3, 128, 128)  # Example input shape (channels, height, width)
num_classes = nb_classes  # Number of output classes

model = SimpleCNN(input_shape[0], num_classes)
print(model)




# model = CombinedModel(input_shape, num_classes).to(device)
# print(model)

# # Training loop with accuracy and classified output
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.01)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

# # Set the number of epochs
# num_epochs = 1

# # Training loop with accuracy and classified output
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.01)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

# overall_progress_bar = tqdm(range(num_epochs), desc="Training Progress")
# for epoch in overall_progress_bar:
#     running_loss = 0.0
#     correct = 0
#     total = 0
#     all_labels = []
#     all_predictions = []

#     # Training phase
#     model.train()
#     for batch, (images_area, images_bar, labels) in enumerate(train_dataloader):
#         optimizer.zero_grad()

#         # Move data to device
#         images_area, images_bar, labels = images_area.to(device), images_bar.to(device), labels.to(device)

#         # Convert labels to LongTensor
#         labels = labels.long()

#         outputs = model(images_area, images_bar)
#         loss = criterion(outputs, labels)
#         running_loss += loss.item()

#         loss.backward()
#         optimizer.step()

#         # Calculate accuracy
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

#         # Collect labels and predictions for output
#         all_labels.extend(labels.cpu().numpy())
#         all_predictions.extend(predicted.cpu().numpy())

#     # Calculate training accuracy
#     epoch_accuracy = accuracy_fn(torch.tensor(all_labels), torch.tensor(all_predictions))

#     # Divide total train loss by length of train dataloader (average loss per batch per epoch)
#     epoch_loss = running_loss / len(train_dataloader)

#     # Print training metrics every 100 epochs
#     if (epoch + 1) % 100 == 0:
#         print(f"Epoch: {epoch+1}/{num_epochs}")
#         print(f'Training Loss: {epoch_loss:.4f} | Training Accuracy: {epoch_accuracy:.2f}%')

#     # Testing phase
#     model.eval()
#     test_loss = 0.0
#     correct = 0
#     total = 0
#     all_labels = []
#     all_predictions = []
#     with torch.inference_mode():
#         for images_area, images_bar, labels in test_dataloader:
#             # Move data to device
#             images_area, images_bar, labels = images_area.to(device), images_bar.to(device), labels.to(device)
            
#             labels = labels.long()
#             outputs = model(images_area, images_bar)
#             loss = criterion(outputs, labels)
#             test_loss += loss.item()

#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()

#             # Collect labels and predictions for output
#             all_labels.extend(labels.cpu().numpy())
#             all_predictions.extend(predicted.cpu().numpy())

#     # Calculate test accuracy
#     test_accuracy = accuracy_fn(torch.tensor(all_labels), torch.tensor(all_predictions))

#     test_loss /= len(test_dataloader)

#     # Print test metrics every 100 epochs
#     if (epoch + 1) % 100 == 0:
#         print(f'Test Loss: {test_loss:.4f} | Test Accuracy: {test_accuracy:.2f}%')

#     # Step the scheduler
#     scheduler.step(test_loss)

Loaded 100 samples from ECG/ECG_TRAIN.ts
Loaded 100 samples from ECG/ECG_TEST.ts
SimpleCNN(
  (layer_stack): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Flatten(start_dim=1, end_dim=-1)
    (13): Linear(in_features=32768, out_f